## Movielens 영화 추천 실습

In [177]:
# MF 모델 학습 방법을 토대로, 내가 좋아할만한 영화 추천 시스템을 제작해 보겠습니다.
# 이번에 활용할 데이터셋은 추천시스템의 MNIST라고 부를만한 Movielens 데이터입니다.

# 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
# 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해볼 수 있습니다.
# 별점을 시청횟수로 해석해서 생각하겠습니다.
# 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.

## 1) 데이터 준비와 전처리

In [11]:
# Movielens 데이터는 rating.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있습니다.

In [188]:
import pandas as pd
import os
data_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
data_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
data = pd.read_csv(data_file_path, sep='::', names=data_cols, engine='python')
orginal_data_size = len(data)

In [189]:
# 필요없는 항목은 삭제한다.
data.drop('timestamp', axis=1, inplace=True)

In [190]:
data.head()

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [191]:
# 3점 이상만 남깁니다.
data = data[data['rating']>=3]
filtered_data_size = len(data)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [192]:
# rating 컬럼의 이름을 count로 바꿉니다.
data.rename(columns={'rating':'count'}, inplace=True)

In [193]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [194]:
# 여기까지가 전처리입니다. 이후에는 이전 스텝에 소개했던 것과 동일한 방식으로 
# MF model을 구성하여 내가 좋아할 만한 영화를 추천해 볼 수 있습니다.

## 2) 분석해 봅시다.

In [195]:
# ratings에 있는 유니크한 영화 개수
# pandas.DataFrame.nunique()은 특정 컬럼에 포함된 유니크한 데이터의 개수를 알아보는데 유용합니다.
data['movie_id'].nunique() # 유저 수

3628

In [196]:
# rating에 있는 유니크한 사용자 수
data['user_id'].nunique()

6039

In [197]:
# 가장 인기있는 영화 30개(인기순)
# 영화제목과 같이 보여주기 위해서 movies에서 영화제목을 가져옴
# data1 = data.copy()
data = pd.merge(data, movies[['movie_id', 'title']], how='left', on='movie_id')

In [198]:
data.head()

,user_id,movie_id,count,title
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975)
1,1,661,3,James and the Giant Peach (1996)
2,1,914,3,My Fair Lady (1964)
3,1,3408,4,Erin Brockovich (2000)
4,1,2355,5,"Bug's Life, A (1998)"


In [199]:
data.groupby(['movie_id', 'title'])['user_id'].count().sort_values(ascending=False)[:30]

movie_id  title                                                
2858      American Beauty (1999)                                   3211
260       Star Wars: Episode IV - A New Hope (1977)                2910
1196      Star Wars: Episode V - The Empire Strikes Back (1980)    2885
1210      Star Wars: Episode VI - Return of the Jedi (1983)        2716
2028      Saving Private Ryan (1998)                               2561
589       Terminator 2: Judgment Day (1991)                        2509
593       Silence of the Lambs, The (1991)                         2498
1198      Raiders of the Lost Ark (1981)                           2473
1270      Back to the Future (1985)                                2460
2571      Matrix, The (1999)                                       2434
480       Jurassic Park (1993)                                     2413
2762      Sixth Sense, The (1999)                                  2385
608       Fargo (1996)                                             2371


In [200]:
# 가장 영화를 많이 본 유저 30명(시청 건수가 높은 순)
top_user_count = data.groupby('user_id')['movie_id'].count()
top_user_count.sort_values(ascending=False).head(30)

user_id
4169    1968
4277    1715
1680    1515
3618    1146
1015    1145
5831    1136
2909    1119
1941    1116
424     1106
1980    1054
3224    1034
1285    1034
3539    1029
3391    1012
3032    1003
1181     981
3841     976
4344     973
4448     970
1088     966
549      956
889      943
4725     936
678      933
1448     930
2063     906
3292     897
1698     883
4808     868
2116     855
Name: movie_id, dtype: int64

In [201]:
# 유저별 몇 개의 영화를 보는지에 대한 통계
user_count = data.groupby('user_id')['movie_id'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: movie_id, dtype: float64

In [202]:
# 유저별 영화 시청 횟수 중앙값에 대한 통계
user_median = data.groupby('user_id')['count'].median()
user_median.describe()

count    6039.000000
mean        4.055970
std         0.432143
min         3.000000
25%         4.000000
50%         4.000000
75%         4.000000
max         5.000000
Name: count, dtype: float64

## 3) 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.

### 내 데이터로 테스트해보기

In [203]:
# 내가 좋아하는 영화 제목 넣어보기
my_favorite = ['Fargo (1996)' , 'Toy Story (1995)' ,'Braveheart (1995)' ,'Sixth Sense, The (1999)' ,'Pulp Fiction (1994)']

# 영화제목을 movie_id로 매핑하기
movie_title = dict(zip(data['title'], data['movie_id']))
movie_ids = [movie_title[i] for i in my_favorite]
movie_ids

# 6041이라는 user_id가 위 영화를 5회씩 시청했다고 입력하기
my_playlist = pd.DataFrame({'user_id': [6041]*5, 'movie_id': movie_ids , 'title': my_favorite, 'count':[5]*5})

if not data.isin({'user_id':[6041]})['user_id'].any():  # user_id에 'yesica'이라는 데이터가 없다면
    data = data.append(my_playlist, ignore_index=True)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

data.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,count,title
836473,6040,1090,3,Platoon (1986)
836474,6040,1094,5,"Crying Game, The (1992)"
836475,6040,562,5,Welcome to the Dollhouse (1995)
836476,6040,1096,4,Sophie's Choice (1982)
836477,6040,1097,4,E.T. the Extra-Terrestrial (1982)
836478,6041,608,5,Fargo (1996)
836479,6041,1,5,Toy Story (1995)
836480,6041,110,5,Braveheart (1995)
836481,6041,2762,5,"Sixth Sense, The (1999)"
836482,6041,296,5,Pulp Fiction (1994)


### 모델에 활용하기 위한 전처리 (실습)¶

In [204]:
# pandas.DataFrame.unique()은 특정 컬럼에 포함된 유니크한 데이터만 모아 줍니다. indexing 작업을 위해 매우 유용합니다.
# 고유한 유저, 영화를 찾아내는 코드
user_unique = data['user_id'].unique()
movie_unique = data['movie_id'].unique()

# 유저, 영화를 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [205]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx[6041])    # 6042명의 유저 중 마지막으로 추가된 유저이니 6041이 나와야 합니다. 
print(movie_to_idx[608])

6039
51


In [210]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = data['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(data):
    print('movie_id column indexing OK!!')
    data['movie_id'] = temp_movie_data
else:
    print('movie_id column indexing Fail!!')

data

user_id column indexing Fail!!
movie_id column indexing Fail!!


,user_id,movie_id,count,title
0,0,0,5,One Flew Over the Cuckoo's Nest (1975)
1,0,1,3,James and the Giant Peach (1996)
2,0,2,3,My Fair Lady (1964)
3,0,3,4,Erin Brockovich (2000)
4,0,4,5,"Bug's Life, A (1998)"
...,...,...,...,...
836478,6039,51,5,Fargo (1996)
836479,6039,40,5,Toy Story (1995)
836480,6039,87,5,Braveheart (1995)
836481,6039,38,5,"Sixth Sense, The (1999)"


### 사용자의 명시적/암묵적 평가

In [211]:
# 우리의 데이터셋에서 1회만 영화를 시청한 데이터의 비율은 어떻게 될까요?
# 1회만 영화를 본 데이터의 비율을 보는 코드
only_one = data[data['count']<2]
one, all_data = len(only_one), len(data)
print(f'{one},{all_data}')
print(f'Ratio of only_one over all data is {one/all_data:.2%}')  # f-format에 대한 설명은 https://bit.ly/2DTLqYU

0,836483
Ratio of only_one over all data is 0.00%


## 4) CSR matrix를 직접 만들어 봅시다.

In [212]:
# csr_matrix((data, (row_ind, col_ind)), [shape=(M, N)])  
# where data, row_ind and col_ind satisfy the relationship a[row_ind[k], col_ind[k]] = data[k]., M,N은 matrix의 shape

# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_movie = data['movie_id'].nunique()

num_user, num_movie

(6040, 3628)

In [216]:
csr_data = csr_matrix((data['count'].values, (data.user_id, data.movie_id)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [217]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [218]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [219]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [220]:
# 모델 훈련
als_model.fit(csr_data_transpose)

## 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [221]:
yesica, Fargo = user_to_idx[6041], movie_to_idx[608]
yesica_vector, Fargo_vector = als_model.user_factors[yesica], als_model.item_factors[Fargo]

In [222]:
yesica_vector

array([-0.16488263,  1.176319  ,  0.9780884 ,  0.5727459 , -0.85594267,
        0.1490854 ,  0.35072693,  0.38342205, -0.02831853, -0.7503371 ,
       -0.63606864,  0.51921517,  0.9685    ,  0.9115704 , -0.22984248,
        0.21618722,  0.22570054,  0.1315583 ,  0.35224348,  0.7236882 ,
       -1.2790223 ,  0.4202572 ,  0.29829672,  0.38763896, -0.27837694,
       -0.5931358 ,  0.13478047, -0.09324267,  0.54142183,  0.15049918,
        0.3803342 , -0.8422951 , -0.2353816 , -0.43654105,  0.511035  ,
        0.16447955, -0.1511802 ,  0.52897984, -0.7061633 ,  0.35434183,
        0.0133774 ,  0.13487798, -0.35694125, -0.47536036, -0.3821228 ,
       -0.01097148, -0.02593108,  0.53365636, -1.0368928 ,  0.15705575,
        0.41405278,  0.42924228,  0.5560368 , -0.01476278, -0.03002081,
        0.28893054,  0.2931865 ,  0.09230138, -0.3962647 ,  0.39271858,
        0.01256442,  0.6689221 , -0.42348385, -0.69488084, -0.46460524,
        0.5354885 ,  0.56490034, -0.13584976,  0.8434961 , -0.98

In [223]:
Fargo_vector

array([ 1.16606755e-03,  2.77287252e-02,  2.39316635e-02,  5.12286201e-02,
       -9.26168449e-03,  1.83614958e-02, -1.35864294e-03,  3.16420905e-02,
       -1.63923912e-02, -2.25875583e-02,  2.64852606e-02,  1.89270396e-02,
        2.73514260e-02,  2.78679449e-02,  3.65713658e-03,  5.95804816e-03,
        2.96585355e-02,  1.11156618e-02,  7.14458758e-03,  3.10008675e-02,
       -1.52748972e-02,  3.07944734e-02,  3.10818460e-02,  6.16730098e-03,
       -1.28974216e-02, -5.87135740e-03, -4.14693588e-03,  4.17749211e-03,
        1.55043965e-02,  1.00940317e-02,  2.48854794e-02, -2.82888152e-02,
       -2.09007803e-02, -1.19201541e-02,  2.94336770e-02,  1.71399266e-02,
       -1.08630601e-02,  1.07907439e-02, -1.19377030e-02,  1.36915250e-02,
        1.11536905e-02,  2.06462992e-03, -1.20577030e-02, -5.57661010e-03,
        5.16470755e-05,  1.07182460e-02,  3.13803367e-02,  9.74109303e-03,
       -2.36969367e-02,  6.48263562e-03,  1.24688409e-02,  4.70260129e-05,
       -1.58822238e-02,  

In [224]:
# zimin과 black_eyed_peas를 내적하는 코드
np.dot(yesica_vector, Fargo_vector)

0.6131736

In [226]:
# 모델이 zimin의 queen에 대한 선호도를 어떻게 예측할지 한 번 보겠습니다
Braveheart = movie_to_idx[110]
Braveheart_vector = als_model.item_factors[Braveheart]
np.dot(yesica_vector, Braveheart_vector)

0.53444606

In [227]:
favorite_movie = 1
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(40, 0.1902218),
 (50, 0.15083227),
 (322, 0.12322411),
 (4, 0.12026137),
 (33, 0.11020111),
 (110, 0.09783815),
 (20, 0.08365149),
 (330, 0.08225842),
 (255, 0.07824427),
 (10, 0.07712605),
 (126, 0.07200107),
 (16, 0.06581121),
 (34, 0.06474488),
 (32, 0.06321691),
 (160, 0.062858716)]

In [228]:
#artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

[1,
 3114,
 34,
 2355,
 588,
 1265,
 2321,
 364,
 1923,
 595,
 2396,
 2687,
 1907,
 1566,
 356]

## 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

## 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.